# Optimization and Simulation with DR

## 課題1

### 1. ライブラリ類のインポートとファイルのロード

In [ ]:
%matplotlib inline

import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from pandas import DataFrame
from scipy.interpolate import griddata
import datarobot as dr

import japanize_matplotlib

if not os.getenv("DATAROBOT_NOTEBOOK_IMAGE"):
    print("not running in DataRobot Notebook")
    from dotenv import load_dotenv
    load_dotenv("../.env", override=True)

client = dr.Client()

seed = 71
np.random.seed(seed)
# sns.set_theme(style='darkgrid')
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.precision", 8)

In [ ]:
df = pd.read_csv("../data/opt_steel_strength.csv")

In [ ]:
y_train = df["降伏強度"]
X_train = df.drop(["ID", "降伏強度", "引張強度"], axis=1)

In [ ]:
# 312rows, 16columns
df.shape

In [ ]:
df.head()

### 2. 探索空間の指定とデータの生成

In [ ]:
%%time
# 10,000件に増やしてもう一回やってみます。
n = 10000
X_artificial = DataFrame(index=range(n),
                         columns=X_train.columns)

for col in X_artificial.columns:

    norm_dist = np.random.normal(X_train[col].mean(), X_train[col].std(), size=n)
    min_value = X_train[col].min()
    max_value = X_train[col].max()
    norm_dist = np.clip(norm_dist, min_value * 0.8, max_value * 1.2)

    X_artificial.loc[:, col] = norm_dist
        
X_artificial = pd.to_numeric(X_artificial.stack(), errors='coerce').unstack()

In [ ]:
X_artificial.to_csv(
    "../data/opt_steel_strength_homework_10000.csv", index=False, encoding="utf-8-sig"
)  # , encoding='SJIS'

### タスク1
生成したDataFrameを用いて、DataRobotのバッチ予測で`引張強度`を予測してみましょう。結果をExcelやpandas等でソートすることで、最も強度が高くなる条件が求まります。<br>
＊予測するモデルを間違えないでください

In [ ]:
deployment_id = (
    "XXXX ここにデプロイIDを記入してください XXXX"  # 引張強度のデプロイメントIDを指定してください
)
# 予測を実行
_, df_result = dr.models.BatchPredictionJob.score_pandas(
    deployment=deployment_id, df="XXXX ここにインプットデータを記入してください XXXX"
)
# 結果から引張強度の最大値を取得
print(f"引張強度の最大値: {df_result.XXXX ここに引張強度の最大値が分かる関数を記入してください XXXX}")

### タスク2
`チタン`と`アルミニウム`の分布を可視化してみましょう

In [ ]:
# 元のデータの分布をみてみましょう。ここではチタンとアルミニウムについて可視化してみます。
feat_1 = "XXXX ここの特徴量名を記入してください XXXX"  # チタン

feat_2 = "XXXX ここの特徴量名を記入してください XXXX"  # アルミニウム

r, p = stats.pearsonr(X_train[feat_1], X_train[feat_2])
g = sns.JointGrid(data=X_train, x=feat_1, y=feat_2)
g.plot_joint(sns.kdeplot, fill=True, color="r")
g.plot_joint(plt.scatter, c="r", s=0.01)
g.plot_marginals(sns.kdeplot, color="r")
g.ax_joint.text(x=1.5, y=2, s=f"pearsonr={r:.2f}, p={p:.2e}")
g.set_axis_labels(feat_1, feat_2)

plt.show()

In [ ]:
# 正規分布から生成したデータの分布をみてみましょう。
r, p = stats.pearsonr(X_artificial[feat_1], X_artificial[feat_2])
g = sns.JointGrid(data=X_artificial, x=feat_1, y=feat_2)
g.plot_joint(sns.kdeplot, fill=True, color="r")
g.plot_joint(plt.scatter, c="r", s=0.01)
g.plot_marginals(sns.kdeplot, color="r")
g.ax_joint.text(x=1.5, y=2, s=f"pearsonr={r:.4f}, p={p:.2F}")
g.set_axis_labels(feat_1, feat_2)

plt.show()

### タスク3
予測結果の分布を可視化してみましょう

In [ ]:
# 特徴空間（2D）における引張強度予測値の分布を可視化してみます
x = np.linspace(X_artificial[feat_1].min(), X_artificial[feat_1].max(), 10)
y = np.linspace(X_artificial[feat_2].min(), X_artificial[feat_2].max(), 10)

# 2Dグリッドの作成
z = griddata(
    (X_artificial[feat_1].values, X_artificial[feat_2].values),
    df_result["XXXX ここのカラム名を記入してください XXXX"].values,
    (x[None, :], y[:, None]),
    method="linear",
)

# プロット
plt.figure(figsize=[7, 7])
plt.contourf(x, y, z, 5, alpha=0.6, cmap=plt.cm.magma)
plt.colorbar()
plt.clim(
    df_result["XXXX ここのカラム名を記入してください XXXX"].min(),
    df_result["XXXX ここのカラム名を記入してください XXXX"].max(),
)
plt.show()